In [1]:
# define Spark client

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .config(
            "spark.sql.extensions",
            "org.projectnessie.spark.extensions.NessieSparkSessionExtensions, org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
        )
        .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
        .config("spark.sql.catalog.iceberg.type", "rest")
        .config("spark.sql.catalog.iceberg.uri", "http://nessie:19120/iceberg/main/")
        .config("spark.sql.catalog.iceberg.ref", "main")
        .config("spark.sql.catalog.iceberg.cache-enabled", False)
        .getOrCreate()
)

spark.sparkContext.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# clean the table if it already exists

spark.sql("""
DROP TABLE IF EXISTS iceberg.default.bank_transfers
""")

DataFrame[]

In [3]:
# create the default namespace in the catalog

spark.sql("""
CREATE NAMESPACE IF NOT EXISTS iceberg.default
""")

DataFrame[]

In [4]:
# create the table we will use in further excercises

spark.sql("""
CREATE TABLE IF NOT EXISTS iceberg.default.bank_transfers (
    id bigint COMMENT 'transfer id',
    amount int COMMENT 'transferred amount, expressed in cents',
    transferred_from string COMMENT 'initiator of the transfer',
    transferred_to string COMMENT 'receiver of the transfer',
    timestamp timestamp COMMENT 'time of the transfer'
)
PARTITIONED BY (years(timestamp))
TBLPROPERTIES (
    'format-version'='2',
    'gc.enabled'='true'
)
""")

DataFrame[]

In [5]:
# describe the created table

spark.sql("""
DESCRIBE FORMATTED iceberg.default.bank_transfers
""").show(50, False)

+----------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------+
|col_name                    |data_type                                                                                                                                                                                                                                                                                           |comment                               |
+----------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
# insert data into the table using SQL

spark.sql("""
INSERT INTO iceberg.default.bank_transfers VALUES
    (1, 12000, "ACME INC",   "ASTROCORP",  TIMESTAMP"2022-11-14T00:55:00"),
    (2, 24000, "John Doe",   "Jane Doe",   TIMESTAMP"2022-11-15T02:11:00"),
    (3,   500, "Deborah S.", "Michael C.", TIMESTAMP"2022-11-17T16:25:07")
""")

# BONUS: Try inserting more data from this or another cell

DataFrame[]

In [7]:
# insert more data to the table, using the DataFrame API

from datetime import datetime
from pyspark.sql import Row
from pyspark.sql.functions import to_timestamp


df = spark.createDataFrame([
    Row(
        id=4,
        amount=200,
        transferred_from="CTX Inc.",
        transferred_to="XYZ GmbH",
        timestamp=datetime.fromisoformat("2022-12-01T07:32:18")
    ),
])

df.writeTo("iceberg.default.bank_transfers").append()

In [12]:
# check the correctness of the inserted data

spark.sql("""
SELECT *
FROM iceberg.default.bank_transfers
ORDER BY id
""").show()

+---+------+----------------+--------------+-------------------+
| id|amount|transferred_from|transferred_to|          timestamp|
+---+------+----------------+--------------+-------------------+
|  1| 12000|        ACME INC|     ASTROCORP|2022-11-14 00:55:00|
|  1| 12000|        ACME INC|     ASTROCORP|2022-11-14 00:55:00|
|  1| 12000|        ACME INC|     ASTROCORP|2022-11-14 00:55:00|
|  2| 24000|        John Doe|      Jane Doe|2022-11-15 02:11:00|
|  2| 24000|        John Doe|      Jane Doe|2022-11-15 02:11:00|
|  2| 24000|        John Doe|      Jane Doe|2022-11-15 02:11:00|
|  3|   500|      Deborah S.|    Michael C.|2022-11-17 16:25:07|
|  3|   500|      Deborah S.|    Michael C.|2022-11-17 16:25:07|
|  3|   500|      Deborah S.|    Michael C.|2022-11-17 16:25:07|
|  4|   200|        CTX Inc.|      XYZ GmbH|2022-12-01 06:32:18|
+---+------+----------------+--------------+-------------------+



In [ ]:
# delete some data and verify that the table does not contain it anymore

spark.sql("""
DELETE FROM iceberg.default.bank_transfers
WHERE id=1
""")

spark.sql("""
SELECT *
FROM iceberg.default.bank_transfers
""").show()

In [ ]:
# query the table again, this time using Dataframe API

df = spark.table("iceberg.default.bank_transfers")
df.show()

In [ ]:
# list table partitions

spark.sql("""
SELECT partition, record_count
FROM iceberg.default.bank_transfers.partitions
""").show(truncate=False)

In [8]:
# query the history of table commits

spark.sql("""
SELECT *
FROM iceberg.default.bank_transfers.history
""").show(truncate=False)

+-----------------------+-------------------+---------+-------------------+
|made_current_at        |snapshot_id        |parent_id|is_current_ancestor|
+-----------------------+-------------------+---------+-------------------+
|2025-06-15 18:39:01.612|6001046170819865776|NULL     |true               |
+-----------------------+-------------------+---------+-------------------+



In [13]:
# query existing snapshots

spark.sql("""
SELECT snapshot_id, manifest_list
FROM iceberg.default.bank_transfers.snapshots
""").show(truncate=False)

+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|snapshot_id        |manifest_list                                                                                                                                                   |
+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|7442410653940799215|s3://warehouse/iceberg/default/bank_transfers_88f92d35-91d3-40ae-9c14-f6c35a078cd9/metadata/snap-7442410653940799215-1-c13b7dd7-2a53-4ff0-ba02-b40b40ad9e8f.avro|
+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+



In [ ]:
# query existing manifests

spark.sql("""
SELECT path
FROM iceberg.default.bank_transfers.manifests
""").show(truncate=False)

In [ ]:
# query metadata log entries

spark.sql("""
SELECT file
FROM iceberg.default.bank_transfers.metadata_log_entries
""").show(truncate=False)